Mnist Classifier using Multi Layer Perceptron

In [ ]:
import torch
import torch.nn.init
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

1. Define Neural Net

In [ ]:
# Multi Layer Perceptron
class Net(nn.Module):
  
  def __init__(self):
    super(Net, self).__init__()
    self.l1 = nn.Linear(784,520)
    self.l2 = nn.Linear(520,320)
    self.l3 = nn.Linear(320,240)
    self.l4 = nn.Linear(240,120)
    self.l5 = nn.Linear(120,10)

  def forward(self, x):
    x = x.view(-1,784) # Flatten data
    # Neural Network
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    return self.l5(x)

net = Net()

print(net)

2. Define Hyperparameters, Loss function, Optimizer

In [ ]:
import torch.optim as optim
# Set Hyperparameter
learning_rate = 0.001
training_epochs = 20
batch_size = 100
valid_size = 0.2

# Set Loss, Optimizer
criterion = nn.CrossEntropyLoss() # softmax + cross entropy
optimizer = optim.Adam(net.parameters(), lr=learning_rate) # ADAM optimizer

3. Load Dataset for training and testing

In [ ]:
# Download datasets
mnist_train = datasets.MNIST(root='data', # download location
                          train=True,
                          transform=transforms.ToTensor(), # convert to tensor
                          download=True)

mnist_test = datasets.MNIST(root='data',
                         train=False, 
                         transform=transforms.ToTensor(),
                         download=True)

4. Divide training dataset into training dataset and validation dataset

In [ ]:
# Prepare dataset for train, validation, test
# divide data between train set and validation set
num_train = len(mnist_train)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_index, valid_index = indices[split:], indices[:split]

# define sampler
train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(valid_index)


train_loader = torch.utils.data.DataLoader(mnist_train, 
                                           batch_size = batch_size, 
                                           sampler = train_sampler)
valid_loader = torch.utils.data.DataLoader(mnist_train, 
                                           batch_size = batch_size,
                                           sampler = valid_sampler)
test_loader = torch.utils.data.DataLoader(mnist_test, 
                                          batch_size = batch_size)

5. Train and evaluate the model(Neural Network)

In [ ]:
for epoch in range(training_epochs):
    # monitor loss
    train_loss = 0
    valid_loss = 0
    
    # train the model   
    net.train()
    for data,label in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forwarding inputs
        output = net(data)
        # calculate the loss
        loss = criterion(output,label)
        # backward propagation
        loss.backward()
        # update parameter
        optimizer.step()
        # update training loss
        train_loss += loss.item() * data.size(0)
            
    # validate the model
    net.eval()
    for data,label in valid_loader:
        # forwarding inputs
        output = net(data)
        # calculate the loss
        loss = criterion(output,label)
        # update validation loss 
        valid_loss = loss.item() * data.size(0)
    
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))   

6. Test the model(Neural Network)

In [ ]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
# Make list for containing correct and total items for each class
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

net.eval()
for data, target in test_loader:
    # forwarding inputs
    output = net(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each class
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        
# print the test accuracy
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))